## Roll-out on a Full Image

### Introduction
Applying machine learning models to entire, full-sized images—especially in the realm of image processing—presents a distinct set of challenges and opportunities. Such a "roll-out" doesn't just involve stretching a model's capabilities across larger pixel dimensions; it tests the model's capacity to consistently and correctly generate outputs, be it segmentation or classification maps, across varying regions of an image.

### Preparation
- Here, we need to process the image to be rolled-out into the shape that is compatible to our model input shape. For example, the model input shape is `(3, 3, 21)`. The below code is for processing the data.


In [ ]:
import pyproj
#import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from math import pi

def WGS84toEASE2(lon, lat):
    proj_EASE2 = pyproj.Proj("+proj=laea +lon_0=0 +lat_0=90 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs")
    proj_WGS84 = pyproj.Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ")
    x , y = pyproj.transform(proj_WGS84, proj_EASE2, lon, lat)
    return x, y

directory = folder_name
directory = '/lustre/home/zcakwc1/project/03/' + directory + '/' + directory
#directory = "S3A_OL_1_EFR____20190301T153919_20190301T153953_20190302T194901_0033_042_054_1620_LN1_O_NT_002.SEN3"
# Load in geolocation
geolocation = netCDF4.Dataset(directory+'/geo_coordinates.nc')
lat = geolocation.variables['latitude'][:]
lon = geolocation.variables['longitude'][:]

    # Load in radiance
Band_Oa01 = netCDF4.Dataset(directory+'/Oa01_radiance.nc')
Oa01_Radiance = Band_Oa01.variables['Oa01_radiance'][:]

X, Y = WGS84toEASE2(lon, lat)

OLCI_file_p=directory
instrument_data = netCDF4.Dataset(OLCI_file_p+'/instrument_data.nc')
solar_flux = instrument_data.variables['solar_flux'][:]
solar_flux_Band_Oa01 = solar_flux[0] # Band 1 has index 0 ect.
detector_index = instrument_data.variables['detector_index'][:]

    # Load in tie geometries
tie_geometries = netCDF4.Dataset(OLCI_file_p+'/tie_geometries.nc')
SZA = tie_geometries.variables['SZA'][:]

Bands=[]
Patches=[]
nx=X.shape[0]-2
ny=X.shape[1]-2
q = 0
for i in range(1,22):
        solar_flux_Band_Oa01 = solar_flux[q]
        print(i)
        bandnumber = '%02d' % (i)
        Band_Oa_temp = netCDF4.Dataset(directory+'/Oa'+bandnumber+'_radiance.nc')

        width = instrument_data.dimensions['columns'].size
        height = instrument_data.dimensions['rows'].size

        TOA_BRF = np.zeros((height, width), dtype='float32')
        angle=np.zeros((TOA_BRF.shape[0],TOA_BRF.shape[1]))
        for x in range(TOA_BRF.shape[1]):
            angle[:,x]=SZA[:,int(x/64)]

        width = instrument_data.dimensions['columns'].size
        height = instrument_data.dimensions['rows'].size

        #TOA_BRF = np.zeros((height, width), dtype=float)
        #TOA_BRF=np.pi*Oa01_Radiance/solar_flux_Band_Oa01[detector_index]/np.cos(np.radians(angle))
        oa = Band_Oa_temp.variables['Oa'+bandnumber+'_radiance'][:]
        TOA_BRF = np.zeros((height, width), dtype=float)
        TOA_BRF=np.pi*np.asarray(oa)/solar_flux_Band_Oa01[detector_index]/np.cos(np.radians(angle))
        Bands.append(TOA_BRF)
        Patches.append(image.extract_patches_2d(np.array(TOA_BRF), (3, 3)).reshape(nx,ny,3,3))
        q = q + 1

Pathches_array=np.asarray(Patches)
x_test_all=np.moveaxis(Pathches_array,0,-1).reshape(Pathches_array.shape[1]*Pathches_array.shape[2],3,3,21)

### Model Application
In this phase, the trained model processes the entire image, generating outputs that classify the different regions into respective categories such as sea-ice and leads. Let's say your saved model is called `model`.

In [ ]:
# Make predictions on the full image
y_pred=model.predict(x_test_all, batch_size = 250)
y_pred1 = np.argmax(y_pred,axis = 1)
# Reshape it for display
map1=y_pred1.reshape(Pathches_array.shape[1], Pathches_array.shape[2])

### Display Results
Visual inspection plays a crucial role here. By juxtaposing the model-generated classified images against the original, we can discern the model's aptitude in retaining the intrinsic spatial structures and patterns of the original image.


In [ ]:
# Alter the view setting
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
# Show the map
plt.imshow(map1)

  Below is a full code of roll-out in a loop manner. A list of filenames of the files that contain the OLCI netCDF files is needed. The below code is a loop that go through each OLCI image that you want to roll-out. You can edit the list of filenames as required. 

In [ ]:
def WGS84toEASE2(lon, lat):
    proj_EASE2 = pyproj.Proj("+proj=laea +lon_0=0 +lat_0=90 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs")
    proj_WGS84 = pyproj.Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ")
    x , y = pyproj.transform(proj_WGS84, proj_EASE2, lon, lat)
    return x, y

txt_file = "path/filename.txt"

# Open the text file and read the folder names
with open(txt_file, 'r') as f:
    folder_names = f.read().splitlines()

# Now folder_names is a list of folder names
# You can loop through it as follows:
q = 0
for folder_name in folder_names:

    # Perform your operations here
    directory = folder_name


    directory = '/lustre/home/zcakwc1/project/03/' + directory + '/' + directory
#directory = "S3A_OL_1_EFR____20190301T153919_20190301T153953_20190302T194901_0033_042_054_1620_LN1_O_NT_002.SEN3"
# Load in geolocation
    geolocation = netCDF4.Dataset(directory+'/geo_coordinates.nc')
    lat = geolocation.variables['latitude'][:]
    lon = geolocation.variables['longitude'][:]

    # Load in radiance
    Band_Oa01 = netCDF4.Dataset(directory+'/Oa01_radiance.nc')
    Oa01_Radiance = Band_Oa01.variables['Oa01_radiance'][:]

    X, Y = WGS84toEASE2(lon, lat)

    OLCI_file_p=directory
    instrument_data = netCDF4.Dataset(OLCI_file_p+'/instrument_data.nc')
    solar_flux = instrument_data.variables['solar_flux'][:]
    solar_flux_Band_Oa01 = solar_flux[0] # Band 1 has index 0 ect.
    detector_index = instrument_data.variables['detector_index'][:]

    # Load in tie geometries
    tie_geometries = netCDF4.Dataset(OLCI_file_p+'/tie_geometries.nc')
    SZA = tie_geometries.variables['SZA'][:]

    Bands=[]
    Patches=[]
    nx=X.shape[0]-2
    ny=X.shape[1]-2
    q = 0
    for i in range(1,22):
        solar_flux_Band_Oa01 = solar_flux[q]
        print(i)
        bandnumber = '%02d' % (i)
        Band_Oa_temp = netCDF4.Dataset(directory+'/Oa'+bandnumber+'_radiance.nc')

        width = instrument_data.dimensions['columns'].size
        height = instrument_data.dimensions['rows'].size

        TOA_BRF = np.zeros((height, width), dtype='float32')
        angle=np.zeros((TOA_BRF.shape[0],TOA_BRF.shape[1]))
        for x in range(TOA_BRF.shape[1]):
            angle[:,x]=SZA[:,int(x/64)]

        width = instrument_data.dimensions['columns'].size
        height = instrument_data.dimensions['rows'].size

        #TOA_BRF = np.zeros((height, width), dtype=float)
        #TOA_BRF=np.pi*Oa01_Radiance/solar_flux_Band_Oa01[detector_index]/np.cos(np.radians(angle))
        oa = Band_Oa_temp.variables['Oa'+bandnumber+'_radiance'][:]
        TOA_BRF = np.zeros((height, width), dtype=float)
        TOA_BRF=np.pi*np.asarray(oa)/solar_flux_Band_Oa01[detector_index]/np.cos(np.radians(angle))
        Bands.append(TOA_BRF)
        Patches.append(image.extract_patches_2d(np.array(TOA_BRF), (3, 3)).reshape(nx,ny,3,3))
        q = q + 1

    Pathches_array=np.asarray(Patches)
    x_test_all=np.moveaxis(Pathches_array,0,-1).reshape(Pathches_array.shape[1]*Pathches_array.shape[2],3,3,21)

    y_pred=model1.predict(x_test_all, batch_size = 250)
    y_pred1 = np.argmax(y_pred,axis = 1)

    map1=y_pred1.reshape(Pathches_array.shape[1], Pathches_array.shape[2])

    np.save('map{}'.format(q),map1)
    q = q + 1